# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

A. Hughes  ->  A. Hughes  |  ['A. Hughes']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
L. Boogaard  ->  L. Boogaard  |  ['L. Boogaard']
A. Gould  ->  A. Gould  |  ['A. Gould']


X. Zhang  ->  X. Zhang  |  ['X. Zhang']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
M. Smith  ->  M. Smith  |  ['M. Smith']
Arxiv has 69 new papers today
          9 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/9 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2307.13009


extracting tarball to tmp_2307.13009...

 done.
Retrieving document from  https://arxiv.org/e-print/2307.13023


extracting tarball to tmp_2307.13023...

...

 done.
Retrieving document from  https://arxiv.org/e-print/2307.13696


/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3508: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3508: LatexWarning: Found documentclass in tmp_2307.13672/aanda.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'latex_table.tex' from 'tmp_2307.13672/latex_table.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


extracting tarball to tmp_2307.13696...

 done.


/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3508: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3508: LatexWarning: Found documentclass in tmp_2307.13696/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2307.13355-b31b1b.svg)](https://arxiv.org/abs/arXiv:2307.13355) | **Stellar angular momentum of disk galaxies at z = 0.7 in the MAGIC survey  I. Impact of the environment**  |
|| W. Mercier, et al. -- incl., <mark>L. Boogaard</mark> |
|*Appeared on*| *2023-07-26*|
|*Comments*| *23 pages, including 5 appendices, 14 figures, accepted version before language correction*|
|**Abstract**| Aims: At intermediate redshift, galaxy groups/clusters are thought to impact galaxies (e.g. their angular momentum). We investigate whether the environment has an impact on the galaxies' angular momentum and identify underlying driving physical mechanisms. Methods: We derive robust estimates of the stellar angular momentum using Hubble Space Telescope (HST) images combined with spatially resolved ionised gas kinematics from the Multi-Unit Spectroscopic Explorer (MUSE) for a sample of ~200 galaxies in groups and in the field at z~0.7 drawn from the MAGIC survey. Using various environmental tracers, we study the position of the galaxies in the the angular momentum-stellar mass (Fall) relation as a function of environment. Results: We measure a 0.12 dex (2sigma significant) depletion of angular momentum for low-mass galaxies (M* < 10^10 Msun) in groups with respect to the field. Massive galaxies located in dense environments have less angular momentum than expected from the low-mass Fall relation but, without a comparable field sample, we cannot infer whether this effect is mass- or environmentally-driven. Furthermore, massive galaxies are found in the centre of the structures and have low systemic velocities. The observed depletion of angular momentum at low mass does not appear linked with the strength of the over-density around the galaxies but it is strongly correlated with the galaxies' systemic velocity normalised by the dispersion of their host group and with their ionised gas velocity dispersion. Conclusions: Group galaxies seem depleted in angular momentum, especially at low mass. Our results suggest that this depletion might be induced by physical mechanisms that scale with the systemic velocity of the galaxies (e.g. stripping or merging) and that such mechanism might be responsible for enhancing the velocity dispersion of the gas as galaxies lose angular momentum. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2307.13009-b31b1b.svg)](https://arxiv.org/abs/arXiv:2307.13009) | **Deep learning denoising by dimension reduction: Application to the  ORION-B line cubes**  |
|| L. Einig, et al. -- incl., <mark>A. Hughes</mark> |
|*Appeared on*| *2023-07-26*|
|*Comments*| **|
|**Abstract**| Context. The availability of large bandwidth receivers for millimeter radio telescopes allows the acquisition of position-position-frequency data cubes over a wide field of view and a broad frequency coverage. These cubes contain much information on the physical, chemical, and kinematical properties of the emitting gas. However, their large size coupled with inhomogenous signal-to-noise ratio (SNR) are major challenges for consistent analysis and interpretation.Aims. We search for a denoising method of the low SNR regions of the studied data cubes that would allow to recover the low SNR emission without distorting the signals with high SNR.Methods. We perform an in-depth data analysis of the 13 CO and C 17 O (1 -- 0) data cubes obtained as part of the ORION-B large program performed at the IRAM 30m telescope. We analyse the statistical properties of the noise and the evolution of the correlation of the signal in a given frequency channel with that of the adjacent channels. This allows us to propose significant improvements of typical autoassociative neural networks, often used to denoise hyperspectral Earth remote sensing data. Applying this method to the 13 CO (1 -- 0) cube, we compare the denoised data with those derived with the multiple Gaussian fitting algorithm ROHSA, considered as the state of the art procedure for data line cubes.Results. The nature of astronomical spectral data cubes is distinct from that of the hyperspectral data usually studied in the Earth remote sensing literature because the observed intensities become statistically independent beyond a short channel separation. This lack of redundancy in data has led us to adapt the method, notably by taking into account the sparsity of the signal along the spectral axis. The application of the proposed algorithm leads to an increase of the SNR in voxels with weak signal, while preserving the spectral shape of the data in high SNR voxels.Conclusions. The proposed algorithm that combines a detailed analysis of the noise statistics with an innovative autoencoder architecture is a promising path to denoise radio-astronomy line data cubes. In the future, exploring whether a better use of the spatial correlations of the noise may further improve the denoising performances seems a promising avenue. In addition, |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2307.13023-b31b1b.svg)](https://arxiv.org/abs/arXiv:2307.13023) | **Limits on dark matter annihilation in prompt cusps from the isotropic  gamma-ray background**  |
|| M. S. Delos, et al. |
|*Appeared on*| *2023-07-26*|
|*Comments*| *11 pages, 10 figures*|
|**Abstract**| Recent studies indicate that thermally produced dark matter will form highly concentrated, low-mass cusps in the early universe that often survive until the present. While these cusps contain a small fraction of the dark matter, their high density significantly increases the expected gamma-ray flux from dark matter annihilation, particularly in searches of large angular regions. We utilize 14 years of Fermi-LAT data to set strong constraints on dark matter annihilation through a detailed study of the isotropic gamma-ray background, excluding with 95% confidence dark matter annihilation to $b\bar{b}$ final states for dark matter masses below 120 GeV. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2307.13398-b31b1b.svg)](https://arxiv.org/abs/arXiv:2307.13398) | **The missing radial velocities of Gaia: a catalogue of Bayesian estimates  for DR3**  |
|| A. P. Naik, A. Widmark |
|*Appeared on*| *2023-07-26*|
|*Comments*| *11 pages, 9 figures (excluding appendices). Submitted to MNRAS. RV catalogues available for download at this http URL . Source code, plotting scripts etc. accessible at this https URL*|
|**Abstract**| In an earlier work, we demonstrated the effectiveness of Bayesian neural networks in estimating the missing line-of-sight velocities of Gaia stars, and published an accompanying catalogue of blind predictions for the line-of-sight velocities of stars in Gaia DR3. These were not merely point predictions, but probability distributions reflecting our state of knowledge about each star. Here, we verify that these predictions were highly accurate: the DR3 measurements were statistically consistent with our prediction distributions, with an approximate error rate of 1.5%. We use this same technique to produce a publicly available catalogue of predictive probability distributions for the 185 million stars up to a G-band magnitude of 17.5 still missing line-of-sight velocities in Gaia DR3. Validation tests demonstrate that the predictions are reliable for stars within approximately 7 kpc from the Sun and with distance precisions better than around 20%. For such stars, the typical prediction uncertainty is 25-30 km/s. We invite the community to use these radial velocities in analyses of stellar kinematics and dynamics, and give an example of such an application. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2307.13574-b31b1b.svg)](https://arxiv.org/abs/arXiv:2307.13574) | **Exploring the Equation of State of the Early Universe: Insights from  BBN, CMB, and PTA Observations**  |
|| Z.-C. Zhao, Q.-H. Zhu, S. Wang, <mark>X. Zhang</mark> |
|*Appeared on*| *2023-07-26*|
|*Comments*| *13 pages, 3 figures*|
|**Abstract**| Strong evidence for a gravitational-wave background (GWB) has been reported in the nano-Hertz band. Interpreting the origin of this background to be scalar-induced gravitational waves (SIGWs), we explore the equation of state (EoS) of the early universe by performing Bayes parameter inferences across the big-bang nucleosynthesis (BBN), cosmic microwave background (CMB), and pulsar timing array (PTA) joint observations for the first time. Assuming a monochromatic power spectrum for primordial curvature perturbations, we obtain the spectral amplitude $A\sim10^{-3}-10^{-1}$ and spectral peak frequency $f_\ast\sim10^{-7}-10^{-6}$ Hz. We find that the radiation domination with EoS $w=1/3$ is compatible with the current observational data, the kination domination with EoS $w=1$ is not forbidden, while the early matter domination with EoS $w=0$ is excluded at more than $2\sigma$ confidence level. These results can be tested with future observations. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2307.13672-b31b1b.svg)](https://arxiv.org/abs/arXiv:2307.13672) | **Central radio galaxies in galaxy clusters: joint surveys by eROSITA and  ASKAP**  |
|| K. Böckmann, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2023-07-26*|
|*Comments*| *16 pages, 12 figures, accepted for publication in A&A*|
|**Abstract**| The extended ROentgen Survey with an Imaging Telescope Array (eROSITA) telescope onboard the Spectrum-Roentgen-Gamma (SRG) mission has finished the first eROSITA All-Sky Survey (eRASS:1), and detected 10$^4$ galaxy clusters in the western Galactic hemisphere. In the radio band, the Australian Square Kilometre Array Pathfinder (ASKAP) telescope finished its pilot 1 phase of the project 'Evolutionary Map of the Universe' (EMU) with 220.000 sources in a 270 deg$^2$ field overlapping with eRASS:1. These two surveys are used to study radio-mode Active Galactic Nuclei (AGN) in clusters. In order to understand the efficiency of radio-mode feedback at the centers of galaxy clusters, we relate the radio properties of brightest cluster galaxies (BCG) to the X-ray properties of the host clusters. We identify the central radio sources in eRASS:1 clusters or calculate corresponding upper limits on the radio luminosity. Then, we derive relations between the X-ray properties of the clusters and the radio properties of the corresponding central radio source. We also apply a mid-infrared color criterion using WISE colors to identify AGN. In total we investigate a sample of 75 clusters. We find a statistically significant between the X-ray luminosity of the cluster and the 944 MHz radio luminosity of the corresponding central radio galaxy. There is also a positive trend between the radio power and the largest linear size (LLS) of the radio source. The density and the LLS do not show any correlation. We find that in high luminosity clusters with L_X > $10^{43}$ erg s$^{-1}$ the kinetic luminosity of the radio jets is not longer correlated with the X-ray luminosity and discuss various reasons. We find an anti-correlation between the central cooling time t_cool and the radio luminosity L_R indicating a need for more powerful AGN in clusters with short central cooling times. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2307.13696-b31b1b.svg)](https://arxiv.org/abs/arXiv:2307.13696) | **The Dark Energy Survey Supernova Program: Cosmological Biases from Host  Galaxy Mismatch of Type Ia Supernovae**  |
|| H. Qu, et al. -- incl., <mark>M. Smith</mark> |
|*Appeared on*| *2023-07-26*|
|*Comments*| *Submitted to ApJ*|
|**Abstract**| Redshift measurements, primarily obtained from host galaxies, are essential for inferring cosmological parameters from type Ia supernovae (SNe Ia). Matching SNe to host galaxies using images is non-trivial, resulting in a subset of SNe with mismatched hosts and thus incorrect redshifts. We evaluate the host galaxy mismatch rate and resulting biases on cosmological parameters from simulations modeled after the Dark Energy Survey 5-Year (DES-SN5YR) photometric sample. For both DES-SN5YR data and simulations, we employ the directional light radius method for host galaxy matching. In our SN Ia simulations, we find that 1.7% of SNe are matched to the wrong host galaxy, with redshift difference between the true and matched host of up to 0.6. Using our analysis pipeline, we determine the shift in the dark energy equation of state parameter (Dw) due to including SNe with incorrect host galaxy matches. For SN Ia-only simulations, we find Dw = 0.0013 +/- 0.0026 with constraints from the cosmic microwave background (CMB). Including core-collapse SNe and peculiar SNe Ia in the simulation, we find that Dw ranges from 0.0009 to 0.0032 depending on the photometric classifier used. This bias is an order of magnitude smaller than the expected total uncertainty on w from the DES-SN5YR sample of around 0.03. We conclude that the bias on w from host galaxy mismatch is much smaller than the uncertainties expected from the DES-SN5YR sample, but we encourage further studies to reduce this bias through better host-matching algorithms or selection cuts. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2307.13238-b31b1b.svg)](https://arxiv.org/abs/arXiv:2307.13238) | **Photometric calibration of the Stellar Abundance and Galactic Evolution  Survey (SAGES): Nanshan One-meter Wide-field Telescope g, r, and i band  imaging data**  |
|| K. Xiao, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2023-07-26*|
|*Comments*| *15 pages in Chinese language, 8 figures, Chinese Science Bulletin accepted and published online (this https URL), see main results in Figures 6, 7 and 8*|
|**Abstract**| In this paper, a total of approximately 2.6 million dwarfs were constructed as standard stars, with an accuracy of about 0.01-0.02 mag for each band, by combining spectroscopic data from the Large Sky Area Multi-Object Fiber Spectroscopic Telescope Data Release 7, photometric data from the corrected Gaia Early Data Release 3, and photometric metallicities. Using the spectroscopy based stellar color regression method (SCR method) and the photometric-based SCR method (SCR' method), we performed the relative calibration of the Nanshan One-meter Wide-field Telescope imaging data. Based on the corrected Pan-STARRS DR1 photometry, the absolute calibration was also performed. In the photometric calibration process, we analyzed the dependence of the calibration zero points on different images (observation time), different gates of the CCD detector, and different CCD positions. We found that the stellar flat and the relative gain between different gates depend on time. The amplitude of gain variation in three channels is approximately 0.5%-0.7% relative to the other channel, with a maximum value of 4%. In addition, significant spatial variations of the stellar flat fitting residual are found and corrected. Using repeated sources in the adjacent images, we checked and discovered internal consistency of about 1-2 mmag in all the filters. Using the PS1 magnitudes synthesized by Gaia DR3 BP/RP spectra by the synthetic photometry method, we found that the photometric calibration uniformity is about 1-2 mmag for all the bands, at a spatial resolution of 1.3 degree. A detailed comparison between the spectroscopy-based SCR and photometric-based SCR method magnitude offsets was performed, and we achieved an internal consistency precision of about 2 mmag or better with resolutions of 1.3 degree for all the filters. Which is mainly from the position-dependent errors of the E(B-V) used in SCR' method. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2307.13355.md
    + _build/html/tmp_2307.13355/graphics/selection_new.png
    + _build/html/tmp_2307.13355/graphics/Fall_relation_comp_full.png
    + _build/html/tmp_2307.13355/graphics/momentum_mass_in_R22_normR22.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\todo}[1]{{\textcolor{red}{#1}}}$
$\newcommand{\OII}{[\ion{O}{ii}]}$
$\newcommand{\OIII}{[\ion{O}{iii}]}$
$\newcommand{\Halpha}{H\alpha}$
$\newcommand{\Halphamath}{{\rm{H}}\alpha}$
$\newcommand{\HST}{\textsc{HST}}$
$\newcommand{\HSTACS}{\textsc{HST-ACS}}$
$\newcommand{\zCOSMOS}{\textsc{zCOSMOS}}$
$\newcommand{\VUDS}{\textsc{VUDS}}$
$\newcommand{\MUSE}{\textsc{MUSE}}$
$\newcommand{\MAGIC}{\textsc{MAGIC}}$
$\newcommand{\COSMOS}{\textsc{Cosmos}}$
$\newcommand{\COSMOStwenty}{\textsc{Cosmos2020}}$
$\newcommand{\COSMOSWall}{\textsc{Cosmos-Wall}}$
$\newcommand{\MUSCATEL}{\textsc{MUSCATEL}}$
$\newcommand{\MUSEWIDE}{\textsc{MUSE-Wide}}$
$\newcommand{\HUDF}{\textsc{MUSE-HUDF}}$
$\newcommand{\HDFS}{\textsc{MUSE-HDFS}}$
$\newcommand{\Galfit}{\textsc{Galfit}}$
$\newcommand{\Cigale}{\textsc{Cigale}}$
$\newcommand{\FAST}{\textsc{FAST}}$
$\newcommand{\MPFIT}{\textsc{MPFIT}}$
$\newcommand{\MPFITEXY}{\textsc{MPFITEXY}}$
$\newcommand{\Ltsfit}{\textsc{LtsFit}}$
$\newcommand{\Mocking}{\textsc{MocKinG}}$
$\newcommand{\Camel}{\textsc{Camel}}$
$\newcommand{\Multinest}{\textsc{MultiNest}}$
$\newcommand{\Lephare}{\textsc{LePhare}}$
$\newcommand{\matplotlib}{\textsc{matplotlib}}$
$\newcommand{\scipy}{\textsc{scipy}}$
$\newcommand{\numpy}{\textsc{numpy}}$
$\newcommand{\astropy}{\textsc{astropy}}$
$\newcommand{\NMAGIC}{2730}$
$\newcommand{\NOIISample}{1142}$
$\newcommand{\NMorphoSample}{890}$
$\newcommand{\NKinematicSample}{571}$
$\newcommand{\Nselection}{182}$
$\newcommand{\NselectionOld}{207}$
$\newcommand{\NselectionFull}{156}$
$\newcommand{\NMSSample}{447}$
$\newcommand{\NTFRSample}{146}$</div>



<div id="title">

# Stellar angular momentum of disk galaxies at $z \approx 0.7$ in the $\MAGIC$  survey$\thanks{Table \ref{tab:catalog_CDS} is only available in electronic form at the CDS via anonymous ftp to cdsarc.u-strasbg.fr (130.79.128.5) or via http://cdsweb.u-strasbg.fr/cgi-bin/qcat?J/A+A/}$$\fnmsep$$\thanks{Based on observations made with ESO telescopes at the Paranal Observatory under programs 094.A-0247, 095.A-0118, 096.A-0596, 097.A-0254, 099.A-0246, 100.A-0607, 101.A-0282, 102.A-0327, and 103.A-0563.}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2307.13355-b31b1b.svg)](https://arxiv.org/abs/2307.13355)<mark>Appeared on: 2023-07-26</mark> -  _23 pages, including 5 appendices, 14 figures, accepted version before language correction_

</div>
<div id="authors">

W. Mercier, et al. -- incl., <mark>L. Boogaard</mark>

</div>
<div id="abstract">

**Abstract:** At intermediate redshift, galaxy groups/clusters are thought to impact galaxy properties such as their angular momentum. We investigate whether the environment has an impact on the galaxies' stellar angular momentum and identify underlying driving physical mechanisms. We derive robust estimates of the stellar angular momentum using Hubble Space Telescope ( $\HST$ ) images combined with spatially resolved ionised gas kinematics from the Multi-Unit Spectroscopic Explorer ( $\MUSE$ ) for a sample of $\sim 200$ galaxies in groups and in the field at $z \sim 0.7$ drawn from the MAGIC survey. Using various environmental tracers, we study the position of the galaxies in the the angular momentum-stellar mass (Fall) relation as a function of environment. We measure a $\SI{0.12}{dex}$ ( $2\sigma$ significant) depletion of stellar angular momentum for low-mass galaxies ( $M_\star < \SI{e10}{\Msun}$ ) located in groups with respect to the field. Massive galaxies located in dense environments have less angular momentum than expected from the low-mass Fall relation but, without a comparable field sample, we cannot infer whether this effect is mass- or environmentally-driven. Furthermore, these massive galaxies are found in the central parts of the structures and have low systemic velocities.	The observed depletion of angular momentum at low stellar mass does not appear linked with the strength of the over-density around the galaxies but it is strongly correlated with (i) the systemic velocity of the galaxies normalised by the dispersion of their host group and (ii) their ionised gas velocity dispersion. Galaxies in groups appear depleted in angular momentum, especially at low stellar mass. Our results suggest that this depletion might be induced by physical mechanisms that scale with the systemic velocity of the galaxies (e.g. stripping or merging) and that such mechanism might be responsible for enhancing the velocity dispersion of the gas as galaxies lose angular momentum.

</div>

<div id="div_fig1">

<img src="tmp_2307.13355/graphics/selection_new.png" alt="Fig1" width="100%"/>

**Figure 1. -** Criteria used for the selection of the kinematics sample. The black points represent galaxies selected according to the surface, S/N, and inclination (removing face-on galaxies only) criteria. Removed galaxies are represented as follows and in this specific order: (i) those removed by the inclination criterion (pink squares), (ii) those removed by the S/N criterion among the remaining galaxies (orange downward pointing triangles), (iii) those removed by the surface criterion among the remaining galaxies but that would have been kept by our previous size criterion used in \citet[][blue upward pointing triangles]{mercier_scaling_2022}, and (iv) those removed by the full selection and by our previous size criterion as well (grey dots). We also show galaxies flagged with peculiar kinematics with red contours. The vertical black line shows the surface selection criterion given in Eq. \ref{eq:sample/selection/surface} and the other black lines represent the S/N selection for different disk axis ratios (see Eq.\ref{eq:sample/selection/S/N}). (*fig:sample/selection/selection*)

</div>
<div id="div_fig2">

<img src="tmp_2307.13355/graphics/Fall_relation_comp_full.png" alt="Fig13" width="100%"/>

**Figure 13. -** Fall relation for the kinematics sample using various formalisms. On the top row is shown the angular momentum derived using $\HST$  maps and on the bottom row that derived using an exponential disk model. The leftmost column represents the Fall relation when using the rotation curve derived from the mass modelling and the rightmost column the Fall relation when using a flat model rotation curve (the top-left plot is similar to Fig. \ref{fig:analysis/momentum first plot}). Galaxies selected for the analysis are shown as black points and those flagged with peculiar kinematics with red contours. Other symbols correspond to galaxies removed by the selection (see Fig. \ref{fig:sample/selection/selection} for more details). The typical uncertainty is shown on the bottom of each plot as a black error-bar. The two black lines show the limit of the Fall relation for an exponential disk without DM halo and with two different disk scale lengths of \SI{5}{\kilo\pc}(dashed line) and \SI{15}{\kilo\pc}(plain line, see Eq. \ref{eq:appendix/angular momentum/special/disk/R22}). (*fig:analysis/reliability/Fall relation HST comparison*)

</div>
<div id="div_fig3">

<img src="tmp_2307.13355/graphics/momentum_mass_in_R22_normR22.png" alt="Fig2" width="100%"/>

**Figure 2. -** Fall relation for the entire kinematics sample, using bulge-removed $\HST$  images. Galaxies selected from the kinematics sample are represented with black points and other symbols represent galaxies removed by different selection criteria (see Fig. \ref{fig:sample/selection/selection}). Red contours correspond to galaxies flagged with peculiar kinematics. The typical uncertainty (including systematic uncertainties added when fitting the relation) is shown as an error bar on the bottom right corner. $^\dagger$Galaxies removed by the surface criterion defined in Sect. \ref{sec:sample/selection} but that would have been kept by the size selection criterion used in \citet{mercier_scaling_2022}. (*fig:analysis/momentum first plot*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2307.13355"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

234  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

5  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
